## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [47]:
df = pd.read_csv('top_archs_for_map_2017_and_later.csv')

## Image URL collector

In [50]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [51]:
# df['image_links'] = ''
# links = []
# for url in df['story_link']:
#     try:
#         page = requests.get(url)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         images = soup.find_all('img')
#         counter = 0
#         for image in images:
#             counter += 1
#             if 'jpg' in image['src']:
#                 links.append(image['src'])
#                 if counter == len(images):
#                     links.append('no image found')
#                 break
#     except:
#         links.append('no_image')

# df['image_links'] = links

## Article link formatter

In [52]:
# len(links)

20

In [53]:
# df["short_description"] = df["full_address"].apply(lambda s: shorten(s, width=20, placeholder=""))
# df['remaining_desc'] = df.apply(lambda row : row['full_address'].replace(str(row['short_description']), ''), axis=1)
# df['remaining_desc']

# df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

0     <a href="https://therealdeal.com/miami/2022/04...
1     <a href="https://therealdeal.com/miami/2022/05...
2     <a href="https://therealdeal.com/miami/2022/02...
3     <a href="https://therealdeal.com/miami/2022/07...
4     <a href="https://therealdeal.com/miami/2022/02...
5     <a href="https://therealdeal.com/miami/2022/08...
6     <a href="https://therealdeal.com/miami/2022/03...
7     <a href="https://therealdeal.com/miami/2022/07...
8     <a href="https://therealdeal.com/miami/2022/05...
9     <a href="https://therealdeal.com/miami/2022/06...
10    <a href="https://therealdeal.com/miami/2022/04...
11    <a href="https://therealdeal.com/miami/2022/06...
12    <a href="https://therealdeal.com/miami/2022/06...
13    <a href="https://therealdeal.com/miami/2022/05...
14    <a href="https://therealdeal.com/miami/2022/07...
15    <a href="https://therealdeal.com/miami/2022/01...
16    <a href="https://therealdeal.com/miami/2022/02...
17    <a href="https://therealdeal.com/miami/202

In [54]:
# df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/04/05/steve-witkoff-flips-waterfront-miami-beach-home-lot-for-188-jump/" target="_blank" rel="noopener noreferrer">6455 Pine Tree Drive</a> Circle, Miami Beach, FL'

## Google Maps API Geolocater Setup

In [48]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [49]:
df['geo_address'] = df['full_address'] + ' Chicago, IL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [6]:
df.columns

Index(['Unnamed: 0', 'name', 'title', 'ID', 'permit_num', 'permit_type',
       'review_type', 'app_start_date', 'issue_date', 'processing_time',
       'street_num', 'street_dir', 'street_name', 'street_suffix', 'work_desc',
       'building_fee_paid', 'zoning_fee_paid', 'other_fee_paid',
       'subtotal_paid', 'building_fee_unpaid', 'zoning_fee_unpaid',
       'other_fee_unpaid', 'subtotal_unpaid', 'building_fee_waived',
       'zoning_fee_waived', 'other_fee_waived', 'total_fee', 'reported_cost',
       'ward', 'full_address', 'app_year_start', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

## Correction section

In [59]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

## HTML popup formatter

In [8]:
df.columns

Index(['Unnamed: 0', 'name', 'title', 'ID', 'permit_num', 'permit_type',
       'review_type', 'app_start_date', 'issue_date', 'processing_time',
       'street_num', 'street_dir', 'street_name', 'street_suffix', 'work_desc',
       'building_fee_paid', 'zoning_fee_paid', 'other_fee_paid',
       'subtotal_paid', 'building_fee_unpaid', 'zoning_fee_unpaid',
       'other_fee_unpaid', 'subtotal_unpaid', 'building_fee_waived',
       'zoning_fee_waived', 'other_fee_waived', 'total_fee', 'reported_cost',
       'ward', 'full_address', 'app_year_start', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [59]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    architect = df['name'].iloc[i]
    work_desc = df['work_desc'].iloc[i]
    issue_date = df['issue_date'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Architect: <strong>{}'''.format(architect) + '''</strong><br>
    Description: <strong>{}'''.format(work_desc) + '''</strong><br>
    Issue Date: <strong>{}'''.format(issue_date) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [62]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [60]:
df.geo_address.isnull().value_counts()

False    659
Name: geo_address, dtype: int64

In [61]:
df[df.isnull().any(axis=1)]

,Unnamed: 0,name,title,ID,permit_num,permit_type,review_type,app_start_date,issue_date,processing_time,...,reported_cost,ward,full_address,app_year_start,geo_address,loc,point,lat,lon,altitude
25,25,WALSH MARK R,ARCHITECT,3127710.0,100878289,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2021-05-06,06/07/2021,32.0,...,7000000.0,NaN,234 W POLK ST,2021.0,"234 W POLK ST Chicago, IL","(234 W Polk St, Chicago, IL 60605, USA, (41.87...","(41.87205220000001, -87.63369709999999, 0.0)",41.872052,-87.633697,0.0
338,338,WALSH MARK R,ARCHITECT,360571.5,100901622,PERMIT - NEW CONSTRUCTION,DIRECT DEVELOPER SERVICES,2021-08-09,09/13/2021,35.0,...,130000000.0,NaN,234 W POLK ST,2021.0,"234 W POLK ST Chicago, IL","(234 W Polk St, Chicago, IL 60605, USA, (41.87...","(41.87205220000001, -87.63369709999999, 0.0)",41.872052,-87.633697,0.0


In [62]:
df = df.dropna(subset=['full_address'])

In [63]:
df.to_csv('data_with_geo_points.csv')

In [76]:
import folium
from folium.plugins import FastMarkerCluster
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Chicago\'s most influential architects')

marker_cluster = MarkerCluster().add_to(m)

for i, r in df.iterrows():
    location = (r['lat'], r['lon'])
    info = (r['full_address'],r['name'],r['work_desc'],r['issue_date'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[0]}{bold_end}{new_line}\
    Architect: {bold_start}{info[1]}{bold_end}{new_line}\
    Description: {bold_start}{info[2]}{bold_end}{new_line}\
    Issue Date: {bold_start}{info[3]}{bold_end}{new_line}'
    folium.Marker(location=location,tooltip=text).add_to(marker_cluster)

# for i in range(0,len(df)):
#     html = popup_html(i)
#     iframe = branca.element.IFrame(html=html)
#     popup = folium.Popup(folium.Html(html, script=True))
#     folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
#                  popup=popup).add_to(m)

#     folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
#                  ).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [77]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [78]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [79]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/south_florida_top_resi_flips_jan_2022_august_2022
